In [52]:
import re
import random
import string


class spellRule(object):
	"""This class is used to add some wrong spell rules"""

	def lyIssue(seg):
		res = []
		if(seg[-2:] != 'ly'):
			return res
		else:
			newSeg = seg[:-2] + 'ies'
			res.append(newSeg)
			return res

	def doubleToSingleIssue(seg):
		res = []
		Len = len(seg)
		pos = []
		for i in xrange(0,Len):
			if(i < len - 1 and seg[i+1] == seg[i]):
				pos.append(i)
		for p in pos:
			newSeg = seg[:p] + seg[p + 1:]
			res.append(newSeg)
		return res


	def missingCharIssue(seg):
		Len = len(seg)
		res = []
		for i in range(0, Len):
			p = random.randint(0, Len - 1)
			newSeg = seg[:p] + seg[p+1:]
			res.append(newSeg)
		return res

	def surplusIssue(seg):
		Len = len(seg)
		res = []

		for i in range(0, Len):
			p = random.randint(0, Len - 1)
			newSeg = seg[:p] + seg[p] + seg[p+1:]
			res.append(newSeg)
		return res

	def randomIssue(seg):
		Len = len(seg)
		res = []

		for i in range(0, Len):
			p = random.randint(0, Len - 1)
			c = random.choice(string.ascii_lowercase)
			newSeg = seg[:p] + c + seg[p+1:]
			res.append(newSeg)
		return res

	def edIssue(seg):
		Len = len(seg)
		res = []
		if seg[-2:] == 'ed':
			newSeg = seg[:-3]
			res.append(newSeg)

		else:
			newSeg = seg + 'ed'
			res.append(newSeg)
		return res

	def addRule(fun, name):
		fmap[name] = fun
        
	fmap ={
		'ly' : lyIssue,
		'doubleToSingle' : doubleToSingleIssue,
		'missing' : missingCharIssue,
		'surplus' : surplusIssue,
		'random' : randomIssue
	}

	def __init__(self):
		super(spellRule, self).__init__()

In [63]:
definitions = [
	'SignatureKeyBlob',
	'RedirectedSessionID',
	'earlyCapabilityFlags',
	'ExeOrFileLength',
	'rdpNegRsp',
	'PDUSource',
	'BlockSizeInBytes',
	'NetworkId',
	'RegionLegacy',
	'IpamApiUserDoesNotHavePermissionToEditIPAddreses',
	'DesiredUsage',
	'GroupID',
	'Hostname'
]


In [98]:

class genFakeWordPair:
	caseIssue = {}
	missingIssue = {}
	spellIssue = {}
	definitons = []
	spellIssueRule = spellRule()
	def readFile(filename):
		with open(filename, 'rt') as f:
			for name in f:
				self.definitons.append(name[:-2])

	def readWord(self,word):
		self.definitons.append(word)


	def readList(self, words):
		self.definitons.extend(words)

	def isLowCase(self, Char):
		return Char.islower()

	def genWordSeg(self, word):

		wordLen = len(word)
		wordSeg = []
		lo = 0
		for i in range(0, wordLen):
			if  self.isLowCase(word[i]) is False:
				if i + 1 < wordLen and self.isLowCase(word[i+1]) is False:
					abbreviation = ""
					while i < wordLen and self.isLowCase(word[i]) is False:
						abbreviation += word[i]
						i += 1
					wordSeg.append(abbreviation)
					lo = i
					continue
				else:
					if( i == 0 ): continue
					seg = word[lo:i]
					wordSeg.append(seg)
					lo = i
			else:
				i += 1
		wordSeg.append(word[lo:])
		return wordSeg


				
	def genCaseIssue(self):
		self.caseIssue = {}
		for word in self.definitons:
			wordSeg = self.genWordSeg(word)
			if len(wordSeg) == 1:
				continue
			else:
				word0 = wordSeg[0]
				if len(word0) == 0:
					return
				if self.isLowCase(word0[0]):
					word0 = word0[0].upper() + word0[1:]
				tmpword = []
				self.caseIssue[word] = []
				self.__genCaseIssue(wordSeg, tmpword, word, 0)


	def __genCaseIssue(self, wordSeg, tmpword, word, idx):
		if len(tmpword) == len(wordSeg):
			data = ""
			for part in tmpword:
 				data += part
			self.caseIssue[word].append(data)
			return
		for i in range(idx,len(wordSeg)):
			tmpword.append(wordSeg[i])
			self.__genCaseIssue(wordSeg, tmpword, word,i + 1)
			tmpword.pop()
			wordSeg[i] = wordSeg[i][0].lower() + wordSeg[i][1:]
			tmpword.append(wordSeg[i])
			self.__genCaseIssue(wordSeg, tmpword,word, i + 1)
			tmpword.pop()

	def genMissingIssue(self):
		self.missingIssue = {}
		for word in self.definitons:
			wordSeg = self.genWordSeg(word)
			self.missingIssue[name] = []
			for i in range(0, len(wordSeg)):
				name = ""
				for j in range(0, len(wordSeg)):
					if(j == i):
						continue
					else:
						name = name + wordSeg[j]
				self.missingIssue[word].append(name)

	def __genSpellIssue(self, wrongSpellDic, tmpword, word, idx):
		if idx == len(wrongSpellDic):
			data = ""
			for part in tmpword:
				data += part
			self.spellIssue[word].append(data)
			return
		for i in range(0, len(wrongSpellDic[idx])):
			tmpword.append[wrongSpellDic[idx][i]]
			self.__genSpellIssue(self,wrongSpellDic, tmpword, word, idx + 1)
			tmpword.pop()


	def genSpellIssue(self):
		self.spellIssue = {}
		for word in self.definitons:
			wordSeg = self.genWordSeg(word)
			wrongSpellDic = []
			for i in range(0, len(wordSeg)):
				wrongSpellDic.append(genWrongSpell(wordSeg[i]))
			tmpword = []
			self.spellIssue[word] = []
			self.__genSpellIssue(wrongSpellDic, tmpword, word, 0)

	def genWrongSpell(self,seg):
		wrongSpellList = []
		for funName in spellIssueRule.fmap:
			wrongSpellList.extend(spellIssueRule.fmap[funName](seg))
		return wrongSpellList


	def writeToFile(self):
		pass



	def __init__(self):
		self.caseIssue = {}
		self.missingIssue = {}
		self.spellIssue = {}
		self.definitons = []
		self.spellIssueRule = spellRule()

In [26]:
def show(data):
	for definition in data:
		size = len(data[definition])
		if( size > 10 ):
			size = 10
		lst = list(range(size))
		idx = random.sample(lst, size)
		print("==========" + definition + ":")
		for i in range(size):
			print ("\t" + data[definition][idx[i]])

In [95]:
def testCaseIssue():
	fakeGenerator = genFakeWordPair()
	fakeGenerator.readList(definitions)
	fakeGenerator.genCaseIssue()
	fakeGenerator.writeToFile()

	show(fakeGenerator.caseIssue)

In [99]:
testCaseIssue()

IndexError: string index out of range